## Реализовать бинарный классификатор изображений бабочек родов calias и aglais

*  metafile.csv - метафайл всех данных
*  metafile_colias.csv - только colias бабочки
*  metafile_aglais.csv - только aglais бабочки
*  metafile_train.csv - обучающий датасет
*  metafile_test.csv - тестовый датасет

Сами изображения хранятся так:
```
dataset_train/
  aglais/
  colias/
dataset_test/
  aglais/
  colias/
```

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

import os


2024-12-21 16:05:28.734521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 16:05:28.743814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734811528.755444  258965 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734811528.758552  258965 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 16:05:28.770950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

train_dir = 'dataset_train'
test_dir = 'dataset_test'

image_size = (150, 150)
batch_size = 32
epochs = 10

# Используем ImageDataGenerator для загрузки изображений и их аугментации
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Нормализация пикселей (от 0 до 255 -> от 0 до 1)
    rotation_range=40,  # Аугментация: случайные повороты
    width_shift_range=0.2,  # Сдвиг по горизонтали
    height_shift_range=0.2,  # Сдвиг по вертикали
    shear_range=0.2,  # Сдвиг по углу
    zoom_range=0.2,  # Масштабирование
    horizontal_flip=True,  # Горизонтальные отражения
    fill_mode='nearest'  # Заполнение пикселей после аугментации
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 1056 images belonging to 2 classes.
Found 4231 images belonging to 2 classes.


### Настраиваем и компилим модель

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


### Обучаем модель на тренировочном датасете

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


/home/v131v/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.5795 - loss: 0.9075

/home/v131v/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 23s 685ms/step - accuracy: 0.5822 - loss: 0.9002 - val_accuracy: 0.8580 - val_loss: 0.3197
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 691ms/step - accuracy: 0.8371 - loss: 0.4104 - val_accuracy: 0.9039 - val_loss: 0.2665
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 23s 710ms/step - accuracy: 0.8821 - loss: 0.3276 - val_accuracy: 0.9036 - val_loss: 0.2367
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 21s 657ms/step - accuracy: 0.8332 - loss: 0.3645 - val_accuracy: 0.8875 - val_loss: 0.2740
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 24s 733ms/step - accuracy: 0.8512 - loss: 0.3492 - val_accuracy: 0.9086 - val_loss: 0.2262
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 684ms/step - accuracy: 0.8647 - loss: 0.3081 - val_accuracy: 0.9179 - val_loss: 0.2336
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 23s 712ms/step - accuracy: 0.8624 - loss: 0.3320 - val_accuracy: 0.9145 - val_loss: 0.2113
Epoch 8/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 669ms/step - accuracy: 0.8805 - loss: 0.2661 - val_accuracy: 0.891

### Оценка точности

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Точность на тестовых данных: {test_acc}")


133/133 ━━━━━━━━━━━━━━━━━━━━ 14s 102ms/step - accuracy: 0.8908 - loss: 0.2481
Точность на тестовых данных: 0.9000236392021179


### Сохраняем модельку в keras формате

In [ ]:
# Сохранение модели
model.save('butterfly_classifier.keras')
